Computo de alto desempeño
======================

Semana 2
Mayo 10, 2017

**Horacio Tapia** 
(htapia@lania.edu.mx)

LANIA-MCA, 2017

# Primeros pasos
Primero debemos importar el modulo y crear una instancia `Client`:

In [2]:
import ipyparallel as ipp
c = ipp.Client()

## Motores conectados al controlador 
Podemos ver la lista de ids de los motores disponibles. Esta debe coincidir con el numero especificado en la pestana. 

In [3]:
c.ids

[0, 1, 2, 3]

In [4]:
c

Esta forma supone que la coneccion _de facto_ es correcta. Dicha informacion puede verse en `IPYTHONDIR/profile_default/security`

In [5]:
!ls ~/.ipython/profile_default/security

ipcontroller-client.json  ipcontroller-engine.json


In [ ]:
# %load ~/.ipython/profile_default/security/ipcontroller-engine.json
{
  "control": 57245, 
  "task": 57251, 
  "hb_ping": 57248, 
  "mux": 57247, 
  "pack": "json", 
  "hb_pong": 57249, 
  "ssh": "", 
  "key": "205b2199-d826-44d7-97ee-c31ce76fd48b", 
  "registration": 57242, 
  "interface": "tcp://127.0.0.1", 
  "iopub": 57253, 
  "signature_scheme": "hmac-sha256", 
  "unpack": "json", 
  "location": "192.168.100.66"
}

## Ejecucion directa
Para la ejecucion directa usamos un objeto `DirectView` que puede construirse a partir de la lista de acceso al cliente:

In [6]:
c[:]

<DirectView [0, 1, 2, 3]>

In [8]:
"Hola alumnos"

'Hola alumnos'

In [9]:
c[0].apply_sync(lambda : "Hola alumnos") # utiliza solo un motor

'Hola alumnos'

In [10]:
c[:].apply_sync(lambda : "Hola muchos alumnos") # utiliza todos los motores

['Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos']

In [11]:
dview = c[:] # asigna todos los motores a la variable dview
dview.apply_sync(lambda : "Hola muchos alumnos")

['Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos',
 'Hola muchos alumnos']

In [12]:
dview

<DirectView [0, 1, 2, 3]>

In [13]:
dview = c.direct_view() # es equivalente a dview = c[:] 

ID de proceso de cada motor

In [14]:
import os
dview.apply_sync(os.getpid)

[3330, 3333, 3336, 3339]

hostname de la maquina donde reside cada motor

In [21]:
import socket
dview.apply_sync(socket.gethostname)

['htapia', 'htapia', 'htapia', 'htapia']

* Una vista proporciona acceso a un subconjunto de motores disponibles para el cliente
* Las tareas se asignan a los motores usando la vista
* `DirectView` permite enviar tareas de manera explicita a un motor especifico
* `LoadBalancedView` administra la distribucion de tareas y es útil cuando se tienen muchas tareas que toman diferentes tiempos para ejecutarse

## Direct view

Sumemos 10 conjuntos de 3 numeros en paralelo usando todos los motores en la vista

In [24]:
dview.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Ahora sumemos 10 conjuntos de 3 numeros en paralelo usando motores alternados

In [25]:
c[::2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Sumemos 10 conjuntos de 3 numeros usando un motor especifico

In [27]:
c[2].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

## Load balanced view

In [32]:
import numpy as np

In [28]:
lview = c.load_balanced_view()

In [33]:
lview.map_sync(lambda x: sum(x), np.random.random((10,1000000)))

[499740.43194067065,
 500382.91163372039,
 500633.34728730452,
 500189.9299793134,
 499674.74516056204,
 499974.76827501046,
 499865.720136819,
 499764.5739453148,
 499456.94730042142,
 499768.63293633098]

### Map
Python tiene una funcion interna llamada `map` que permite aplicar cualquier funcion lemento a elemente a una secuencia. Para este ejemplo utilizaremos una funcion muy sencilla definida a continuacion.

In [59]:
def cube(x):
    return x**3
cube(2)

8

In [70]:
lista1=[cube(x) for x in range(1,7)]
lista1

[1, 8, 27, 64, 125, 216]

In [68]:
map(cube, range(1,7))

[1, 8, 27, 64, 125, 216]

In [71]:
serial = map(cube, range(1,70))

In [72]:
parallel = dview.map_sync(cube, range(1,70))

In [73]:
serial == parallel

True

In [74]:
%timeit map(cube, range(1,70))
%timeit dview.map_sync(cube, range(1,70))

100000 loops, best of 3: 14.3 µs per loop
10 loops, best of 3: 82.9 ms per loop


In [15]:
dview.map_sync(cube, range(1,7))

[1, 8, 27, 64, 125, 216]

In [22]:
def multi(a,b):
    return a*b
dview.map_sync(multi, [0,1,3,5],[2,4,6,8])

[0, 4, 18, 40]

Definamos una funcion que determina si un numero dado es primo:

In [17]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

In [18]:
PrimeQ(2)

True

In [19]:
PrimeQ(137)

True

In [20]:
PrimeQ(135)

False

# Ejercicios

Determinar si un numero de la forma $2^n -1$, donde $n$ es un entero, es primo o no. Los primos que tienen esta forma se llaman primos de Mersenne.

1. Escribir una funcion que determine si el numero dado por $2^n -1$ es primo (usar la funcion anterior)
2. Seleccionar los primos de Mersenne de entre los primeros 5000 numeros enteros (nota: el resultado es {2, 3, 5, 7, 13, 17, 19, 31, 61, 89, 107, 127, 521, 607, 1279, 2203, 2281, 3217, 4253, 4423}
3. Estimar el tiempo que su computadora tarda en encontrar los primero 5000 primos de Mersenne


# Tarea

1. Estudiar la informacion contenida en el archivo `/.ipython/profile_default/security/ipcontroller-client.json` y explicar en un parrafo que significa. [Leer la documentacion](http://ipyparallel.readthedocs.io/en/latest/intro.html) 
2. Investigar el uso de los metodos `apply`,`apply_sync` de las instancias `DirectView` (el objeto `dview`) y discutir las diferencias con `map` y `map_sync`, por medio de ejemplos. (Ver la documentacion)
3. Determinar que partes de un programa que encuentra los primeros 5000 primos de Mersenne son paralelizables y sugerir como podrian paralelizarlo
4. Aplicar el metodo `apply` y/o `map` descritos aqui al programa que selecciona los primos de Mersenne de entre los primeros 5000 numeros enteros
